###Package installation


In [ ]:
!pip install -U sentence-transformers
!pip install gcld3

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import spacy
import pathlib
import random
import math
import re
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import gcld3

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
import en_core_web_lg
nlp = en_core_web_lg.load()

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Loading txt Data Startups**

In [9]:
path_startup = pathlib.Path("drive/My Drive/Capstone Shared Docs/result/Txt Files Public and Startups/Startups/Startup 2013")

In [10]:
txt_startup =list(path_startup.glob('*.txt'))

In [11]:
all_text_startup = {}

for p in txt_startup:
    n = p.name.split("_")[0]
    file = open(p)
    # Remove first two and last characters, because of ascii encoding
    text = file.read().replace("***///***","")[2:-1]
    # remove ""
    text_list = text.split(" ")
    text_list = [x for x in text_list if x!=""]
    text = " ".join(text_list)
    file.close()
    all_text_startup[n] = text
  
print(len(all_text_startup))

894


In [12]:
Txt_startup_frame = pd.DataFrame.from_dict(all_text_startup, orient='index')
Txt_startup_frame.columns = ['Text']
Txt_startup_frame['companyid'] = Txt_startup_frame.index
Txt_startup_frame.index = np.arange(0,len(Txt_startup_frame.companyid))
Txt_startup_frame = Txt_startup_frame.assign(comp_type='S')
Txt_startup_frame = Txt_startup_frame[['companyid', 'comp_type', 'Text']]

In [13]:
stop_words_l=stopwords.words('english')
Txt_startup_frame['Text_cleaned']=Txt_startup_frame.Text.apply(lambda x: " ".join(re.sub(r'[^a-zA-Z]',' ',w).lower() for w in x.split() if re.sub(r'[^a-zA-Z]',' ',w).lower() not in stop_words_l) )

In [14]:
Txt_startup_frame.head()

,companyid,comp_type,Text,Text_cleaned
0,110847,S,Blog Jobs Company Interested in contributing t...,blog jobs company interested contributing alph...
1,109827,S,Home List Your Boat LOG IN SIGN UP HOW IT WORK...,home list boat log sign works find boat rent c...
2,71885,S,Digital Guardian | Verdasys Your web-browser d...,digital guardian verdasys web browser suppor...
3,73604,S,A Better Way to Work Helping {{ value }} be mo...,better way work helping value successful...
4,106695,S,Products News Blog How It Works Login Store Lo...,products news blog works login store locator f...


# **Loading txt Data Public Companies**

In [18]:
path_public = pathlib.Path("drive/My Drive/Capstone Shared Docs/result/Txt Files Public and Startups/Public Companies/Public 2013")

In [17]:
txt_public =list(path_public.glob('*.txt'))

In [19]:
all_text_public = {}

for p in txt_public:
    n = p.name.split("_")[0]
    file = open(p)
    # Remove first two and last characters, because of ascii encoding
    text = file.read().replace("***///***","")[2:-1]
    # remove ""
    text_list = text.split(" ")
    text_list = [x for x in text_list if x!=""]
    text = " ".join(text_list)
    file.close()
    all_text_public[n] = text

print(len(all_text_public))

9929


In [21]:
Txt_public_frame = pd.DataFrame.from_dict(all_text_public, orient='index')
Txt_public_frame.columns = ['Text']
Txt_public_frame['companyid'] = Txt_public_frame.index
Txt_public_frame.index = np.arange(0,len(Txt_public_frame.companyid))
Txt_public_frame = Txt_public_frame.assign(comp_type='P')
Txt_public_frame = Txt_public_frame[['companyid', 'comp_type', 'Text']]

In [22]:
stop_words_l=stopwords.words('english')
Txt_public_frame['Text_cleaned']=Txt_public_frame.Text.apply(lambda x: " ".join(re.sub(r'[^a-zA-Z]',' ',w).lower() for w in x.split() if re.sub(r'[^a-zA-Z]',' ',w).lower() not in stop_words_l) )

In [ ]:
Txt_public_frame.head()

# **Concatenate Public and Startup DB**

In [23]:
Txt_all_frame = pd.concat([Txt_startup_frame, Txt_public_frame])
Txt_all_frame.index = np.arange(0,len(Txt_all_frame.companyid))

In [ ]:
Txt_all_frame.head()

# **Solving some language issues**

In [24]:
lan = []
rel_lan = []
detector = gcld3.NNetLanguageIdentifier(min_num_bytes=0, max_num_bytes=10000)

for i in range(len(Txt_all_frame['Text_cleaned'])):
  t =Txt_all_frame.Text_cleaned[i]
  t = str(t)
  
  result = detector.FindLanguage(text=t)
  lan.append(result.language)
  rel_lan.append(result.is_reliable)


In [25]:
Txt_all_frame['language'] = lan
Txt_all_frame['Rela_language'] = rel_lan

In [26]:
Txt_all_frame_en = Txt_all_frame[(Txt_all_frame.language=='en') ] #| ( (Txt_all_frame.language!='en') & (Txt_all_frame.language==False) )
Txt_all_frame_en.index = np.arange(0,len(Txt_all_frame_en.companyid))

# **Lenght of text**

In [30]:
Txt_all_frame_en.head()

,companyid,comp_type,Text,Text_cleaned,language,Rela_language
0,110847,S,Blog Jobs Company Interested in contributing t...,blog jobs company interested contributing alph...,en,True
1,109827,S,Home List Your Boat LOG IN SIGN UP HOW IT WORK...,home list boat log sign works find boat rent c...,en,True
2,71885,S,Digital Guardian | Verdasys Your web-browser d...,digital guardian verdasys web browser suppor...,en,True
3,73604,S,A Better Way to Work Helping {{ value }} be mo...,better way work helping value successful...,en,True
4,106695,S,Products News Blog How It Works Login Store Lo...,products news blog works login store locator f...,en,True


# **embeddings**

### embedding for word2vec

In [27]:
import spacy.cli
spacy.cli.download("en_core_web_lg")
nlp = spacy.load("en_core_web_lg", disable=["tagger", "parser", "ner"])

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [31]:
max_characters = 1000000
docs_train = []

for i,r in Txt_all_frame_en.iterrows():
  doc_text = r.Text_cleaned
  text_len = len(doc_text)
  num_of_vectors = math.ceil(text_len/max_characters)
  v = np.zeros((300,1))
  w = 0
  for j in range(num_of_vectors):
    small_text = doc_text[max_characters*(j):max_characters*(j+1)]
    vec = nlp(small_text).vector
    vec = vec.reshape(vec.shape[0],1)
    v = np.sum( np.hstack((v,vec*len(small_text))),axis=1)
    v = v.reshape((v.shape[0],1))
    w += len(small_text)
  v /= w
  docs_train.append(v)
X_train = np.vstack([d.T for d in docs_train])

In [33]:
X_train.shape


(10823, 300)

### embedding for tf-idf


```
# This is formatted as code
```



In [ ]:
pip install gcld3
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import gcld3

In [ ]:
stopWords = set(stopwords.words("english"))
vectorizer = TfidfVectorizer(stop_words = stopWords)

In [ ]:
X_train = vectorizer.fit_transform(Txt_all_frame_en['Text_cleaned'])

### embedding for Bert


In [ ]:
sbert_model = SentenceTransformer('roberta-large-nli-stsb-mean-tokens')

In [ ]:
%%time
docs_train = []

pct = int(np.percentile(Txt_all_frame_en.Text_len, 95))

for i in range(len(Complete_frame)):

  if len(Complete_frame['Text_cleaned'][i]) > pct :
    v = sbert_model.encode(Complete_frame['Text_cleaned'][i][:pct])
    docs_train.append(v)
  
  else:
    v = sbert_model.encode(Complete_frame['Text_cleaned'][i])
    docs_train.append(v)
X_train = np.vstack([d.T for d in docs_train])

# **similarity**

In [34]:
pairwise_similarities=cosine_similarity(X_train)

In [36]:
def get_most_similar(sim_matrix_v, startup_index, how_many=1):
  sim_matrix_copy = sim_matrix_v.copy()
  v = sim_matrix_copy[startup_index,:]
  v[startup_index] = 0
  most_similar_indices = []
  similarity_list = []
  for i in range(how_many):
    ind = np.argmax(v)
    similarity = np.max(v)
    v[ind] = 0
    most_similar_indices.append(ind)
    similarity_list.append(similarity)
  return most_similar_indices, similarity_list

In [38]:
rows_startup = Txt_all_frame_en[Txt_all_frame_en.comp_type == 'S'].index.tolist()

In [42]:
Txt_all_frame_en

,companyid,comp_type,Text,Text_cleaned,language,Rela_language
0,110847,S,Blog Jobs Company Interested in contributing t...,blog jobs company interested contributing alph...,en,True
1,109827,S,Home List Your Boat LOG IN SIGN UP HOW IT WORK...,home list boat log sign works find boat rent c...,en,True
2,71885,S,Digital Guardian | Verdasys Your web-browser d...,digital guardian verdasys web browser suppor...,en,True
3,73604,S,A Better Way to Work Helping {{ value }} be mo...,better way work helping value successful...,en,True
4,106695,S,Products News Blog How It Works Login Store Lo...,products news blog works login store locator f...,en,True
...,...,...,...,...,...,...
10570,5516,P,You are here: home symbol lookup Content Solut...,here home symbol lookup content solutions app...,en,True
10571,12924,P,About Us About Us As the worlds largest asset ...,us us worlds largest asset manager help indiv...,en,True
10572,8064,P,English Japanese z[ j[X[X j[X[X 2007.11.05 A}i...,english japanese z j x x j x x a i...,en,True
10573,7375,P,>Register >Advisor Login >Home >Allianz Worldw...,register advisor login home allianz worldw...,en,True


In [ ]:
Dict_similarity = []
Strat_score = []
n = 5

for i in rows_startup:

  most_similar_index,most_similar_similarity = get_most_similar(pairwise_similarities, i, n)
  print(most_similar_index)
  IDs = [Txt_all_frame_en.companyid[j] for j in most_similar_index]
  dict_sim = {IDs[i]: most_similar_similarity[i] for i in range(len(IDs))} 
  strategy_score = np.mean(1-np.array(most_similar_similarity))

  Dict_similarity.append(dict_sim)
  Strat_score.append(strategy_score)

In [ ]:
startups = Txt_all_frame_en[Txt_all_frame_en.comp_type == 'S']
startups['Dict_similarity'] = Dict_similarity
startups['Strat_score'] = Strat_score

In [ ]:
startups.Strat_score.var()